In [2]:
! pip install datasets evaluate transformers diffusers accelerate ftfy pyarrow --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 3.0 MB/s eta 0:00:00


## Tokenizer

In [3]:
from transformers import AutoTokenizer # class

# tokenizers are specific to the models!
checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
# distilbert/distilbert-base-uncased-finetuned-sst-2-english
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [4]:
raw_inputs = [
    "I am learning Operating Systems and it's so fun.", # one num
    "I write terrible C++ code!", # sec num
] # text inputs that the model cannot ingest directly
inputs = tokenizer(raw_inputs, padding=True, truncation=True, return_tensors="pt", max_length=15)
print(inputs)

{'input_ids': tensor([[ 101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569,
         1012,  102],
        [ 101, 1045, 4339, 6659, 1039, 1009, 1009, 3642,  999,  102,    0,    0,
            0,    0]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]])}


In [ ]:
[4, 5, 6] # row 1 -> inp 1
[1, 2, <pad_tokens>] # row 2 -> inp 2

# unable to form a matrix -> matmul not possible -> model processing np

In [5]:
# another way to do it - a closer look into the tokenizer
sequence = "I am learning Operating Systems and it's so fun."
tokens = tokenizer.tokenize(sequence)

print("--------TOKENIZED SENTENCE--------")
print(tokens)

print("\n\n--------TOKENS MAPPED TO THEIR IDS--------")
ids = tokenizer.convert_tokens_to_ids(tokens)
print(ids)

--------TOKENIZED SENTENCE--------
['i', 'am', 'learning', 'operating', 'systems', 'and', 'it', "'", 's', 'so', 'fun', '.']


--------TOKENS MAPPED TO THEIR IDS--------
[1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012]


**TAKE A PAUSE!**

**SPECIAL TOKENS!**

In [6]:
# already tokenized input

# mapping the token ids back to the tokens
decoded_string = tokenizer.decode([1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012])
print(decoded_string)

i am learning operating systems and it ' s so fun.


In [7]:
# raw text
decoded_string = tokenizer.decode([101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569,
         1012,  102])
print(decoded_string)

[CLS] i am learning operating systems and it ' s so fun. [SEP]


**Padding - ways to do it**

In [8]:
# Will pad the sequences up to the maximum sequence length
model_inputs = tokenizer(sequence, padding="longest")

print("--------TOKEN IDS (after pad the sequences up to the maximum sequence length)--------")
print(model_inputs["input_ids"])

print("\n--------DECODED STRING (after pad the sequences up to the maximum sequence length)--------")
decoded_padded_string = tokenizer.decode(model_inputs["input_ids"])
print(decoded_padded_string)



# Will pad the sequences up to the model max length
# (512 for BERT or DistilBERT) model max length = the max sentence length that the model can ingest
model_inputs = tokenizer(sequence, padding="max_length")



print("--------TOKEN IDS (after Will pad the sequences up to the model max length)--------")
print(model_inputs["input_ids"])

print("\n--------DECODED STRING (after Will pad the sequences up to the model max length)--------")
decoded_padded_string = tokenizer.decode(model_inputs["input_ids"])
print(decoded_padded_string)



# Will pad the sequences up to the specified max length
model_inputs = tokenizer(sequence, padding="max_length", max_length=8)

print("--------TOKEN IDS (after padding to max_length=8)--------")
print(model_inputs["input_ids"])

print("\n--------DECODED STRING (after padding to max_length=8)--------")
decoded_padded_string = tokenizer.decode(model_inputs["input_ids"])
print(decoded_padded_string)

--------TOKEN IDS (after pad the sequences up to the maximum sequence length)--------
[101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012, 102]

--------DECODED STRING (after pad the sequences up to the maximum sequence length)--------
[CLS] i am learning operating systems and it ' s so fun. [SEP]
--------TOKEN IDS (after Will pad the sequences up to the model max length)--------
[101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [9]:
print("--------TOKEN IDS (after padding to max_length=8)--------")
print(model_inputs["input_ids"])

print("\n--------DECODED STRING (after padding to max_length=8)--------")
decoded_padded_string = tokenizer.decode(model_inputs["input_ids"])
print(decoded_padded_string)

--------TOKEN IDS (after padding to max_length=8)--------
[101, 1045, 2572, 4083, 4082, 3001, 1998, 2009, 1005, 1055, 2061, 4569, 1012, 102]

--------DECODED STRING (after padding to max_length=8)--------
[CLS] i am learning operating systems and it ' s so fun. [SEP]


In [13]:
print("--------INDIVIDUAL PREDICTIONS--------")
for i, prediction_idx in enumerate(predictions):
    text = raw_inputs[i]
    sentiment = model.config.id2label[prediction_idx.item()]
    print(f"Input: '{text}'")
    print(f"Prediction: {sentiment} (Index: {prediction_idx.item()})\n")

--------INDIVIDUAL PREDICTIONS--------
Input: 'I am learning Operating Systems and it's so fun.'
Prediction: POSITIVE (Index: 1)

Input: 'I write terrible C++ code!'
Prediction: NEGATIVE (Index: 0)



In [16]:
import torch
import torch.nn.functional as F

# Assuming 'outputs' from the model call is available, which contains outputs.logits

print("--------ORIGINAL LOGIT TENSOR--------")
print(outputs.logits)

# Step 1: Apply Softmax to convert logits to probabilities
# dim=-1 means applying softmax across the last dimension (the class dimension)
probabilities = F.softmax(outputs.logits, dim=-1)
print("\n--------PROBABILITIES (after Softmax)--------")
print(probabilities)

# Step 2: Apply Argmax to get the index of the highest probability
# dim=1 means finding the max along the dimension representing classes
final_predictions = torch.argmax(probabilities, dim=1)
print("\n--------FINAL PREDICTIONS (after Argmax)--------")
print(final_predictions)

# For clarity, let's also show the sentiments
print("\n--------FINAL PREDICTIONS WITH SENTIMENT LABELS--------")
for i, prediction_idx in enumerate(final_predictions):
    text = raw_inputs[i]
    sentiment = model.config.id2label[prediction_idx.item()]
    print(f"Input: '{text}'")
    print(f"Prediction: {sentiment} (Index: {prediction_idx.item()})\n")

--------ORIGINAL LOGIT TENSOR--------
tensor([[-4.3357,  4.6875],
        [ 4.6717, -3.7884]], grad_fn=<AddmmBackward0>)

--------PROBABILITIES (after Softmax)--------
tensor([[1.2056e-04, 9.9988e-01],
        [9.9979e-01, 2.1170e-04]], grad_fn=<SoftmaxBackward0>)

--------FINAL PREDICTIONS (after Argmax)--------
tensor([1, 0])

--------FINAL PREDICTIONS WITH SENTIMENT LABELS--------
Input: 'I am learning Operating Systems and it's so fun.'
Prediction: POSITIVE (Index: 1)

Input: 'I write terrible C++ code!'
Prediction: NEGATIVE (Index: 0)



In [ ]:
print("--------MODEL ID2LABEL MAPPING--------")
print(model.config.id2label)

print("\nBased on this mapping:")
for index, label in model.config.id2label.items():
    print(f"Index {index} corresponds to: {label}")

## Models

In [10]:
from transformers import AutoModel

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModel.from_pretrained(checkpoint)

Loading weights:   0%|          | 0/100 [00:00<?, ?it/s]

DistilBertModel LOAD REPORT from: distilbert-base-uncased-finetuned-sst-2-english
Key                   | Status     |  | 
----------------------+------------+--+-
pre_classifier.bias   | UNEXPECTED |  | 
classifier.bias       | UNEXPECTED |  | 
classifier.weight     | UNEXPECTED |  | 
pre_classifier.weight | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [11]:
outputs = model(**inputs)
print(outputs.last_hidden_state.shape) # bs, seq_len, dim

torch.Size([2, 14, 768])


    "I am learning Operating Systems and it's so fun.", # one num
    "I write terrible C++ code!", # sec num

In [15]:
from transformers import AutoModelForSequenceClassification
import torch

checkpoint = "distilbert-base-uncased-finetuned-sst-2-english"
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)
outputs = model(**inputs)


# there are a total of 2 classes -- use this info to make sense of the shape of logits
print("--------SHAPE OF LOGIT TENSOR--------")
print(outputs.logits.shape)

print("\n\n--------LOGIT TENSOR--------")
print(outputs.logits)

# convert logits to probs --> softmax (math func)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
predictions = torch.argmax(predictions, dim=1)
print("\n\n--------PREDICTIONS--------")
print(predictions) # os one, c++ one


Loading weights:   0%|          | 0/104 [00:00<?, ?it/s]

--------SHAPE OF LOGIT TENSOR--------
torch.Size([2, 2])


--------LOGIT TENSOR--------
tensor([[-4.3357,  4.6875],
        [ 4.6717, -3.7884]], grad_fn=<AddmmBackward0>)


--------PREDICTIONS--------
tensor([1, 0])


--------SHAPE OF LOGIT TENSOR--------

**torch.Size([2, 2])**

The **first 2** represents the **batch size**, which is the number of **input sequences (sentences)** you provided

(in this case, "I am learning Operating Systems..." and "I write terrible C++ code!").

The **second 2** represents the number of classes for the **classification task**.

Since this model is fine-tuned for sentiment analysis **(likely positive/negative)**, it has two possible output classes.



--------LOGIT TENSOR--------

**tensor([[-4.3357,  4.6875],[ 4.6717, -3.7884]], grad_fn=<AddmmBackward0>)**


logits. Logits are the raw, unnormalized scores generated by the **last layer of the model** for each class. They are not probabilities yet.

**Each row corresponds to an input sequence,** and
**each column corresponds to the score for a specific class**:

For the first sentence ("I am learning Operating Systems and it's so fun."), the scores are [-4.3357, 4.6875]. The second score is significantly higher.

For the second sentence ("I write terrible C++ code!"), the scores are [4.6717, -3.7884]. The first score is significantly higher.



--------PREDICTIONS--------

**tensor([1, 0])**

 the final predicted class labels after applying a softmax function to the logits (to convert them into probabilities) and then argmax (to select the class with the highest probability). These are the indices of the predicted classes:

**For the first sentence, the prediction is 1** (because 4.6875 at index 1 was the highest logit).
**For the second sentence, the prediction is 0** (because 4.6717 at index 0 was the highest logit).


In [14]:
print("--------INDIVIDUAL PREDICTIONS--------")
for i, prediction_idx in enumerate(predictions):
    text = raw_inputs[i]
    sentiment = model.config.id2label[prediction_idx.item()]
    print(f"Input: '{text}'")
    print(f"Prediction: {sentiment} (Index: {prediction_idx.item()})\n")

--------INDIVIDUAL PREDICTIONS--------
Input: 'I am learning Operating Systems and it's so fun.'
Prediction: POSITIVE (Index: 1)

Input: 'I write terrible C++ code!'
Prediction: NEGATIVE (Index: 0)



**Here's how it works:**

**Logit Scores: **For each input, the model outputs a set of logits (e.g., [-4.3357, 4.6875] for the first sentence). Each number in this set corresponds to the model's raw 'confidence' or 'score' for a specific class.

**Softmax Function:** These raw logit scores are then passed through a softmax function. Softmax converts these raw scores into probabilities that sum up to 1 across all classes. A higher logit score for a class will result in a higher probability for that class after softmax.

**Argmax for Prediction**: After obtaining the probabilities, the argmax function is applied. argmax simply picks the index (or class) that has the highest probability (which corresponds to the highest logit score before softmax). This highest-probability class is then chosen as the model's final prediction for that input.

Identifying the maximum value within the logit scores for each input, as that **maximum value will ultimately lead to the predicted class** after the standard post-processing (softmax and argmax).


The mapping from numerical indices (like 0 and 1) to human-readable labels (POSITIVE, NEGATIVE) is stored within the model's configuration.

This is determined during the model's training (fine-tuning on a specific dataset like SST-2 for sentiment analysis).

You can inspect this mapping directly using **model.config.id2label.** This attribute is a dictionary that shows exactly which index corresponds to which label.

In [18]:
print(model.config.id2label)

{0: 'NEGATIVE', 1: 'POSITIVE'}


**If the softmax output for index 0 is higher, the model predicts a NEGATIVE sentiment.**


**If the softmax output for index 1 is higher, the model predicts a POSITIVE sentiment.**


This mapping is crucial for translating the model's numerical predictions into meaningful human-readable categories.

In [17]:
import torch
import torch.nn.functional as F

# Assuming 'outputs' from the model call is available, which contains outputs.logits

print("--------ORIGINAL LOGIT TENSOR--------")
print(outputs.logits)

# Step 1: Apply Softmax to convert logits to probabilities
# dim=-1 means applying softmax across the last dimension (the class dimension)
probabilities = F.softmax(outputs.logits, dim=-1)
print("\n--------PROBABILITIES (after Softmax)--------")
print(probabilities)

# Step 2: Apply Argmax to get the index of the highest probability
# dim=1 means finding the max along the dimension representing classes
final_predictions = torch.argmax(probabilities, dim=1)
print("\n--------FINAL PREDICTIONS (after Argmax)--------")
print(final_predictions)

# For clarity, let's also show the sentiments
print("\n--------FINAL PREDICTIONS WITH SENTIMENT LABELS--------")
for i, prediction_idx in enumerate(final_predictions):
    text = raw_inputs[i]
    sentiment = model.config.id2label[prediction_idx.item()]
    print(f"Input: '{text}'")
    print(f"Prediction: {sentiment} (Index: {prediction_idx.item()})\n")

--------ORIGINAL LOGIT TENSOR--------
tensor([[-4.3357,  4.6875],
        [ 4.6717, -3.7884]], grad_fn=<AddmmBackward0>)

--------PROBABILITIES (after Softmax)--------
tensor([[1.2056e-04, 9.9988e-01],
        [9.9979e-01, 2.1170e-04]], grad_fn=<SoftmaxBackward0>)

--------FINAL PREDICTIONS (after Argmax)--------
tensor([1, 0])

--------FINAL PREDICTIONS WITH SENTIMENT LABELS--------
Input: 'I am learning Operating Systems and it's so fun.'
Prediction: POSITIVE (Index: 1)

Input: 'I write terrible C++ code!'
Prediction: NEGATIVE (Index: 0)

